# TODO

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
 - 문서가 길면 토큰수 초과로 답변을 생성하지 못할 수 있다.
 - 문서가 길면 (인풋이 길면) 답변 생성에 시간이 너무 많이 소요된다.
 3. 임베딩 -> 벡터 데이터베이스에 저장
 4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [12]:
#%pip install --upgrade --quiet  docx2txt langchain-community

## Pinecone VectorStore를 이용

In [1]:
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.


## Load and Split

In [30]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter

# text splitter는 보통 두개의 파라미터를 가짐. 1. 청크사이즈, 2. 청크오버랩
text_splitter = CharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("tax-markdown.docx")
# document = loader.load()
# document를 loader.load()대신 loader.load_and_split(text_splitter)으로 불러오면 청크사이즈와 청크오버랩을 고려해서 쪼개진 문서를 반환함.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [35]:
len(document_list)

182

In [ ]:
# %pip install -qU langchain_text_splitters

Note: you may need to restart the kernel to use updated packages.


## Embedding

In [31]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')



이제 우리가 임베딩 한 결과를 VectorDB에 넣어주어야함.

### pineconeDB를 이용

In [32]:
import getpass
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

index_name = 'tax-markdown-index'
index = pc.Index(index_name)
# database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name) #이거 공식문서에서 이제 빠짐.
database = PineconeVectorStore( index=index, embedding=embeddings)



In [33]:
# 이제 add_documents함수를 이용해 문서를 추가할 수 있음.
database.add_documents(documents=document_list)

['97eeb0f7-8780-46f0-b02e-bd72bd81ad59',
 '7d821977-6da9-433c-8154-82e795ecc7ad',
 '1dd55a50-4036-49bd-b0f1-f34607e94122',
 '0341748a-5f4c-430e-9a78-15723f992673',
 'afdbb74f-4328-4838-bf59-c72460a2dca2',
 '2378e94f-cc58-467a-be98-999730428de4',
 '9a86fa7a-6958-46e9-9cf6-564ae7cf6eb5',
 '0ffa4516-c2cc-4059-a00e-940316ce3c20',
 '30d40b2d-fe57-4524-829b-e37a54d3c88b',
 'c7459ed4-2f89-4641-aa68-3f2b83319c68',
 'be8568c9-8419-49d5-a93d-3be6512be76b',
 'bc988e70-e831-4ae4-aed5-9eec9ef139a5',
 '68391929-49de-4723-8dbb-1162a1ec8204',
 '23eea962-cb68-445c-93a9-7d14afcf214f',
 'a78fa670-b115-4a2b-b211-1eeb5ebcfe71',
 '3b786f80-061c-4332-b482-968b3e7554d9',
 '97d82c0a-4eae-41ec-a4e0-1dcd8aee4440',
 'a992574d-5fd6-44c1-becc-b97e4574b202',
 'a47dba3c-1ea9-42d9-aa73-23652099501e',
 'c7a9f7a3-a93f-4ab4-9256-0b03fadeade7',
 'c02221c1-5d63-424e-ba49-bd4fb44d258c',
 'b51b5da9-7a07-4fd0-8786-986b4efefa1d',
 'ddd62b2b-0e9a-47f3-85fc-11a0b42262cc',
 'fb50afad-092d-48cc-b672-e34017f33361',
 '0f3e072b-1b84-

쪼개놓은 결과인 document_list를 embedding모델을 통해 VectorDB에 저장

In [34]:
query = "연봉 5000만원의 직장인은 소득세가 얼마인가요?"
# 이건 실무 잡이야기이긴 한데, 존댓말로 물어보면 답변을 더 잘해준다고 합니다. 영어질문은 Please 붙여서 하면 더 답변이 잘나온다는 이야기가 있습니다.
retrieved_docs = database.similarity_search(query, k=5) # similarity_search의 디폴트 값이 4이므로 k값 설정을 안해두면 4개의 문서를 반환할 것임.

In [37]:
document_list[52]

Document(metadata={'source': 'tax-markdown.docx', 'text': '가. 종교단체에 기부한 금액이 있는 경우\n\n\u3000\u3000한도액 = [종합소득금액(제62조에 따른 원천징수세율을 적용받는 이자소득 및 배당소득은 제외한다)에서 제1호에 따른 기부금을 뺀 금액을 말하며, 이하 이 항에서 “소득금액”이라 한다] × 100분의 10 + [소득금액의 100분의 20과 종교단체 외에 기부한 금액 중 적은 금액]\n\n나. 가목 외의 경우\n\n\u3000\u3000한도액 = 소득금액의 100분의 30\n\n⑤ 제1항부터 제3항까지의 규정을 적용할 때 과세기간 종료일 이전에 혼인ㆍ이혼ㆍ별거ㆍ취업 등의 사유로 기본공제대상자에 해당되지 아니하게 되는 종전의 배우자ㆍ부양가족ㆍ장애인 또는 과세기간 종료일 현재 65세 이상인 사람을 위하여 이미 지급한 금액이 있는 경우에는 그 사유가 발생한 날까지 지급한 금액에 제1항부터 제3항까지의 규정에 따른 율을 적용한 금액을 해당 과세기간의 종합소득산출세액에서 공제한다.\n\n⑥ 제1항부터 제4항까지의 규정에 따른 공제는 해당 거주자가 대통령령으로 정하는 바에 따라 신청한 경우에 적용한다.\n\n⑦ 국세청장은 제3항제2호라목에 따른 교육비가 세액공제 대상에 해당하는지 여부를 확인하기 위하여 「한국장학재단 설립 등에 관한 법률」 제6조에 따른 한국장학재단 등 학자금 대출ㆍ상환업무를 수행하는 대통령령으로 정하는 기관(이하 이 항에서 “한국장학재단등”이라 한다)에 학자금대출 및 원리금 상환내역 등 대통령령으로 정하는 자료의 제공을 요청할 수 있다. 이 경우 요청을 받은 한국장학재단등은 특별한 사유가 없으면 그 요청에 따라야 한다.<신설 2016. 12. 20.>\n\n⑧ 제4항에도 불구하고 2024년 1월 1일부터 2024년 12월 31일까지 지급한 기부금을 해당 과세기간의 합산과세되는 종합소득산출세액(필요경비에 산입한 기부금이 있는 경우 사업소득에 대한 산출세액은 제외한다)에서 공제하는 경우에는

In [38]:
retrieved_docs

[Document(id='2378e94f-cc58-467a-be98-999730428de4', metadata={'source': 'tax-markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [40]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}

"""

In [41]:
ai_message = llm.invoke(prompt)

In [42]:
ai_message.content

'연봉 5,000만원의 직장인이 납부해야 할 소득세를 계산해보겠습니다. 소득세는 종합소득 과세표준에 따라 세율이 적용됩니다.\n\n1. **연봉 5,000만원의 경우**:\n   - 5,000만원은 **1,400만원 초과 ~ 5,000만원 이하** 구간에 해당합니다.\n   - 이 구간의 세율은 다음과 같습니다:\n     - 1,400만원 이하: 과세표준 × 6%\n     - 1,400만원 초과 ~ 5,000만원 이하: 84만원 + (1,400만원 초과분 × 15%)\n\n2. **계산**:\n   - 5,000만원에서 1,400만원을 빼면 3,600만원이 남습니다.\n   - 따라서, 세액은 다음과 같이 계산됩니다:\n     - 기본 세액: 84만원\n     - 추가 세액: 3,600만원 × 15% = 540만원\n\n3. **총 소득세**:\n   - 총 소득세 = 기본 세액 + 추가 세액\n   - 총 소득세 = 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5,000만원의 직장인은 약 **624만원**의 소득세를 납부해야 합니다.'

'연봉 5,000만원의 직장인이 납부해야 할 소득세를 계산해보겠습니다. 소득세는 종합소득 과세표준에 따라 세율이 적용됩니다.\n\n1. **연봉 5,000만원의 경우**:\n   - 5,000만원은 **1,400만원 초과 ~ 5,000만원 이하** 구간에 해당합니다.\n   - 이 구간의 세율은 다음과 같습니다:\n     - 1,400만원 이하: 과세표준 × 6%\n     - 1,400만원 초과 ~ 5,000만원 이하: 84만원 + (1,400만원 초과분 × 15%)\n\n2. **계산**:\n   - 5,000만원에서 1,400만원을 빼면 3,600만원이 남습니다.\n   - 따라서, 세액은 다음과 같이 계산됩니다:\n     - 기본 세액: 84만원\n     - 추가 세액: 3,600만원 × 15% = 540만원\n\n3. **총 소득세**:\n   - 총 소득세 = 기본 세액 + 추가 세액\n   - 총 소득세 = 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5,000만원의 직장인은 약 **624만원**의 소득세를 납부해야 합니다.'

## RetrievalQA Chain

In [43]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [44]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [45]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [47]:
## RetrievalQA Chain을 어떻게 사용하는지 알아보자.
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
    
)

In [49]:
ai_message = qa_chain({"query":query})

In [50]:
ai_message

{'query': '연봉 5000만원의 직장인은 소득세가 얼마인가요?',
 'result': '연봉 5000만원의 직장인의 소득세는 소득세법에 따라 계산되며, 구체적인 세액은 개인의 공제 항목 및 세율에 따라 달라집니다. 일반적으로, 소득세는 누진세율로 적용되므로, 정확한 금액을 알기 위해서는 세무 전문가의 상담이 필요합니다. 따라서, 정확한 소득세 금액은 제공된 정보로는 알 수 없습니다.'}